In [ ]:
# mesh loading settingsx
meshes_path = "D:\\knpob\\20220723-Mesh4D\\data\\meshes\\6kmh_braless_26markers"
start=0
stride = 1
end=120

# fe file path
regist_path = "..\\temp\\regist"
# fe_path = "..\\temp\\c10 init"
# fe_path = "..\\temp\\c10 0.00015"
# fe_path = "..\\temp\\c10 0.0001"
# fe_path = "..\\temp\\c10 0.0002"
fe_path = "..\\temp\\traced"

# Load 4D scanning sequence

In [ ]:
from mesh4d import obj3d

mesh_ls, texture_ls = obj3d.load_mesh_series(
    folder=meshes_path,
    start=start,
    stride=stride,
    end=end,
)

In [ ]:
body_ls = obj3d.init_obj_series(
    mesh_ls, 
    obj_type=obj3d.Obj3d_Deform
    )

# Load breast trace

In [ ]:
import os
import numpy as np

breast_trace = np.load(os.path.join(fe_path, 'breast_trace.npy'))
breast_trace.shape

In [ ]:
rot = np.load(os.path.join(regist_path, 'rot.npy'))
scale = np.load(os.path.join(regist_path, 'scale.npy'))
t = np.load(os.path.join(regist_path, 't.npy'))
rot, scale, t

# Graphical evaluation

In [ ]:
plot_num = len(breast_trace)
plot_stride = 10

mesh_num = len(mesh_ls)
mesh_fps = 120
fe_fps = 200

In [ ]:
for fe_idx in range(0, plot_num, plot_stride):
    # convert fe index to mesh index
    mesh_idx = int(fe_idx / fe_fps * mesh_fps)

    if mesh_idx >= mesh_num:
        break

    print(fe_idx, mesh_idx)

In [ ]:
# before alignment
import pyvista as pv
from mesh4d import utils
from mesh4d.analyse import measure

scene = pv.Plotter()
scene.open_movie(os.path.join(fe_path, 'breast_trace' + '.mp4'))

for fe_idx in range(0, plot_num, plot_stride):
    # convert fe index to mesh index
    mesh_idx = int(fe_idx / fe_fps * mesh_fps)

    if mesh_idx >= mesh_num:
        break

    scene.clear()

    points = breast_trace[fe_idx]
    nearest_points = measure.nearest_points_from_plane(mesh_ls[mesh_idx], points)

    cloud = pv.PolyData(points)
    cloud['point_color'] = np.linalg.norm(nearest_points - points, axis=1)
    
    scene.add_points(cloud, point_size=1e-5, opacity=0.5, cmap='cool')
    scene.add_mesh(mesh_ls[mesh_idx], opacity=0.1, show_edges=True)
    scene.camera_position = 'xy'
    scene.write_frame()

    percent = (fe_idx + 1) / plot_num
    utils.progress_bar(percent, back_str=f" exported the {fe_idx}-th frame")

scene.close()

In [ ]:
# before alignment
import pyvista as pv
from mesh4d import utils
from mesh4d.analyse import measure

scene = pv.Plotter()
scene.open_movie(os.path.join(fe_path, 'breast_trace_aligned' + '.mp4'))

for fe_idx in range(0, plot_num, plot_stride):
    # convert fe index to mesh index
    mesh_idx = int(fe_idx / fe_fps * mesh_fps)

    if mesh_idx >= mesh_num:
        break

    scene.clear()

    points = breast_trace[fe_idx]
    points = points + t
    nearest_points = measure.nearest_points_from_plane(mesh_ls[mesh_idx], points)

    cloud = pv.PolyData(points)
    cloud['point_color'] = np.linalg.norm(nearest_points - points, axis=1)
    
    scene.add_points(cloud, point_size=1e-5, opacity=0.5, cmap='cool')
    scene.add_mesh(mesh_ls[mesh_idx], opacity=0.1, show_edges=True)
    scene.camera_position = 'xy'
    scene.write_frame()

    percent = (fe_idx + 1) / plot_num
    utils.progress_bar(percent, back_str=f" exported the {fe_idx}-th frame")

scene.close()

# Quantitative evaluation

In [ ]:
eval_num = len(breast_trace)
eval_stride = 1

mesh_num = len(mesh_ls)
mesh_fps = 120
fe_fps = 200

In [ ]:
for fe_idx in range(0, eval_num, eval_stride):
    # convert fe index to mesh index
    mesh_idx = int(fe_idx / fe_fps * mesh_fps)

    if mesh_idx >= mesh_num:
        break

    print(fe_idx, mesh_idx)

In [ ]:
dist_ls = []

for fe_idx in range(0, eval_num, eval_stride):
    # convert fe index to mesh index
    mesh_idx = int(fe_idx / fe_fps * mesh_fps)

    if mesh_idx >= mesh_num:
        break

    scene.clear()

    points = breast_trace[fe_idx]
    points = points + t
    nearest_points = measure.nearest_points_from_plane(mesh_ls[mesh_idx], points)
    dist_ls.append(np.linalg.norm(nearest_points - points, axis=1))

    percent = (fe_idx + 1) / eval_num
    utils.progress_bar(percent, back_str=f" exported the {fe_idx}-th frame")

In [ ]:
deviation = np.array(dist_ls)
utils.save_pkl_object(deviation, fe_path, 'deviation')
print(f"overall distance: {deviation.mean():.2f} ± {deviation.std():.2f} (mm)")